In [ ]:
#任务: 基于flare文本数据，建立LSTM模型，预测序列文字:
#1.完成数据预处理，将文字序列数据转化为可用于LSTM输入的数据
#2.查看文字数据预处理后的数据结构，并进行数据分离操作
#3.针对字符串输入(" flare is a teacher in ai industry. He obtained his phd in Australia.")，预测其对应的后续字符
#备注: 模型结构: 单层LSTM，输出有20个神经元; 每次使用前20个字符预测第21个字符

In [15]:
#read data
data = open('flare.txt').read()
#移除换行符
data = data.replace('\n','').replace('\r','')
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained h

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM

#找到所有不重复的字符和长度
letters = list(set(data))
print(letters[1])
num_letters = len(letters)
print(num_letters)


c
23


In [33]:
#建立字典
int_to_char = dict((i,c) for i,c in enumerate(letters))
char_to_int = dict((c,i) for i,c in enumerate(letters))
print(int_to_char)
#print(char_to_int)



{0: 'i', 1: 'c', 2: 'b', 3: 'd', 4: 's', 5: 'm', 6: 'u', 7: 'f', 8: 'p', 9: 'o', 10: 't', 11: 'a', 12: '.', 13: 'e', 14: 'r', 15: 'H', 16: 'h', 17: 'A', 18: ' ', 19: 'n', 20: 'S', 21: 'l', 22: 'y'}


In [43]:
time_step = 20

In [44]:
# 数据预处理，将文本转换为LSTM输入数据，采用one-hot编码
def prepare_data_onehot(data, char_to_int, time_step, num_letters):
    X = []
    y = []

    # 生成输入序列和对应的输出
    for i in range(0, len(data) - time_step):
        input_seq = data[i:i + time_step]
        output_char = data[i + time_step]

        seq_ints = [char_to_int[char] for char in input_seq]

        # 将输入序列转换为one-hot矩阵
        seq_onehot = np.zeros((time_step, num_letters))
        for t, char_idx in enumerate(seq_ints):
            seq_onehot[t, char_idx] = 1

        X.append(seq_onehot)
        y.append(char_to_int[output_char])

    X = np.array(X)
    y_one_hot = tf.keras.utils.to_categorical(y, num_classes=num_letters)

    return X, y_one_hot

In [47]:
# 调用函数处理数据
X, y = prepare_data_onehot(data, char_to_int, time_step, num_letters)
print(X.shape)
print(y.shape)

(56148, 20, 23)
(56148, 23)


In [48]:
#数据分离
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [56]:
print(X_train.shape)
print(X_test.shape)


(39303, 20, 23)
(16845, 20, 23)


In [57]:
#set lstm model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
model = Sequential()
model.add(LSTM(20, input_shape=(20, num_letters),activation='relu'))
model.add(Dense(num_letters, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 23)             │           483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,003 (15.64 KB)

 Trainable params: 4,003 (15.64 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
#train model
model.fit(X_train, y_train, epochs=10, batch_size=1000, validation_data=(X_test, y_test))

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.6228 - loss: 1.3155 - val_accuracy: 0.7701 - val_loss: 0.8618
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.8084 - loss: 0.7498 - val_accuracy: 0.9086 - val_loss: 0.4541
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.9020 - loss: 0.4518 - val_accuracy: 0.9586 - val_loss: 0.2803
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.9589 - loss: 0.2364 - val_accuracy: 0.9838 - val_loss: 0.1379
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9876 - loss: 0.1211 - val_accuracy: 1.0000 - val_loss: 0.0611
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.8793 - loss: 0.4882 - val_accuracy: 0.7284 - val_loss: 0.8166
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8517 - loss: 0.4710 - val_accuracy: 0.9911 - val_loss: 0.0690
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9990 - loss: 0.0552 - val_accuracy: 1.0000 - v

In [63]:
#make prediction test accurancy
import numpy as np
from sklearn.metrics import accuracy_score
#train dataset
y_train_pred = model.predict(X_train)
y_train_pred = np.argmax(y_train_pred, axis=1)
y_train_true = np.argmax(y_train, axis=1)
train_acc = accuracy_score(y_train_true, y_train_pred)
print("Train accuracy:", train_acc)
y_test_pred = model.predict(X_test)
#test dataset
y_test_pred = np.argmax(y_test_pred, axis=1)
y_test_true = np.argmax(y_test, axis=1)
test_acc = accuracy_score(y_test_true, y_test_pred)
print("Test accuracy:", test_acc)

1229/1229 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Train accuracy: 1.0
527/527 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Test accuracy: 1.0


In [84]:
# 准备新的测试文本
new_letters = " flare is a teacher in ai industry. He obtained his phd in Australia."
x_new, y_new = prepare_data_onehot(new_letters, char_to_int, time_step, num_letters)

# 模型预测
y_new_pred = model.predict(x_new)
y_new_pred_indices = np.argmax(y_new_pred, axis=1)

# 将预测结果转换为字符
y_new_pred_chars = [int_to_char[i] for i in y_new_pred_indices]

# 将真实标签转换为字符
y_new_true_indices = np.argmax(y_new, axis=1)
y_new_true_chars = [int_to_char[i] for i in y_new_true_indices]

# 打印预测结果和真实值对比
print("预测字符序列:", ''.join(y_new_pred_chars))
print("实际字符序列:", ''.join(y_new_true_chars))

# 逐个对比预测结果和真实值
print("\n详细对比:")
for i in range(len(y_new_pred_chars)):
    input_seq = new_letters[i:i+time_step]
    pred_char = y_new_pred_chars[i]
    true_char = y_new_true_chars[i]
    match = "✓" if pred_char == true_char else "✗"

    print(f"输入: '{input_seq}'")
    print(f"预测: '{pred_char}', 实际: '{true_char}' {match}")
    print("-" * 50)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
预测字符序列: hn ai industry. He obtained his phd in Australia.
实际字符序列: in ai industry. He obtained his phd in Australia.

详细对比:
输入: ' flare is a teacher '
预测: 'h', 实际: 'i' ✗
--------------------------------------------------
输入: 'flare is a teacher i'
预测: 'n', 实际: 'n' ✓
--------------------------------------------------
输入: 'lare is a teacher in'
预测: ' ', 实际: ' ' ✓
--------------------------------------------------
输入: 'are is a teacher in '
预测: 'a', 实际: 'a' ✓
--------------------------------------------------
输入: 're is a teacher in a'
预测: 'i', 实际: 'i' ✓
--------------------------------------------------
输入: 'e is a teacher in ai'
预测: ' ', 实际: ' ' ✓
--------------------------------------------------
输入: ' is a teacher in ai '
预测: 'i', 实际: 'i' ✓
--------------------------------------------------
输入: 'is a teacher in ai i'
预测: 'n', 实际: 'n' ✓
--------------------------------------------------
输入: 's a teacher in ai in'
预测: 'd', 实际: 'd' ✓
-----------------

In [89]:
# 预测并打印结果
for i in range(0, x_new.shape[0]-20):
    print(f"{new_letters[i:i+20]}, --predict next letter is-- {y_new_pred_chars[i]}")

 flare is a teacher , --predict next letter is-- h
flare is a teacher i, --predict next letter is-- n
lare is a teacher in, --predict next letter is--  
are is a teacher in , --predict next letter is-- a
re is a teacher in a, --predict next letter is-- i
e is a teacher in ai, --predict next letter is--  
 is a teacher in ai , --predict next letter is-- i
is a teacher in ai i, --predict next letter is-- n
s a teacher in ai in, --predict next letter is-- d
 a teacher in ai ind, --predict next letter is-- u
a teacher in ai indu, --predict next letter is-- s
 teacher in ai indus, --predict next letter is-- t
teacher in ai indust, --predict next letter is-- r
eacher in ai industr, --predict next letter is-- y
acher in ai industry, --predict next letter is-- .
cher in ai industry., --predict next letter is--  
her in ai industry. , --predict next letter is-- H
er in ai industry. H, --predict next letter is-- e
r in ai industry. He, --predict next letter is--  
 in ai industry. He , --predict